In [1]:

import psycopg2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re


In [2]:
dataset = pd.read_csv(f"SharkAttack.csv", encoding='latin-1')

In [3]:
pd.set_option('display.max_columns', 500) # mostrar todas colunas
dataset

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [5]:
dataset.loc[dataset["Unnamed: 23"].notnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,NaN,"Buttock bitten, tissue removed",N,NaN,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,NaN,Leg bitten by netted shark PROVOKED INCIDENT,N,NaN,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


In [6]:
dataset.loc[dataset["Unnamed: 22"].notnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,31,"No injury, shark bumped surfboard",N,16h00,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN


In [7]:
# Colunas "Unnamed" nao tem dados importantes, dropar
dataset.drop(["Unnamed: 23"], axis=1, inplace=True)
dataset.drop(["Unnamed: 22"], axis=1, inplace=True)

In [8]:
# Criando novo dataset com as "rows" corretas, sem linhas quase inteiras nulas.
datasetMenor = dataset.loc[0:6301]
datasetMenor

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0


In [9]:
#Rename de algumas colunas
datasetMenor = datasetMenor.rename(columns= {"Species ":"Species", "original order": "original_order", "Case Number.1":"case_number.1", "Case Number.2":"case_number.2", "Fatal (Y/N)":"Fatal", "Sex ":"Sex"})

In [10]:
# Ordernar e usar de index coluna "original_order"
datasetMenor.sort_values(["original_order"], inplace=True)

In [11]:
datasetMenor.set_index("original_order", inplace=True)
datasetMenor.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,href formula,href,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,,,,
2.0,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001
3.0,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002
4.0,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003
5.0,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004
6.0,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005


In [12]:
datasetMenor.isnull().sum()

Case Number                  1
Date                         0
Year                         2
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2831
Injury                      28
Fatal                      539
Time                      3354
Species                   2838
Investigator or Source      17
pdf                          0
href formula                 1
href                         0
case_number.1                0
case_number.2                0
dtype: int64

In [13]:
datasetMenor.groupby(["href"]).count()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,href formula,case_number.1,case_number.2
href,,,,,,,,,,,,,,,,,,,,
Q93http://sharkattackfile.net/spreadsheets/pdf_directory/ND-0100-CDDollar.pdf,1,1,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/.2014.04.12.R-inflatable,1,1,1,1,1,0,0,1,1,0,0,1,1,0,1,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/0005.00.00-AustralianAboriginal.pdf,1,1,1,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/1543.00.00.R-LasCasas.pdf,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/1554.00.00-Rondelet.pdf,1,1,1,1,1,1,0,0,1,1,0,0,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2017.08.27-Brundler.pdf,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2018.01.13-Stewart.pdf,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1
http://sharkattackfile.net/spreadsheets/pdf_directory/w014.01.25-Grant.pdf,4,4,4,4,4,4,3,4,4,4,2,4,4,1,3,4,4,4,4,4


In [14]:
datasetMenor["pdf"] = "http://sharkattackfile.net/spreadsheets/pdf_directory/" + datasetMenor["pdf"].astype(str)

In [15]:
# As colunas "pdf", "href formula" e "href" tem a mesma informacao dividida entre elas. A coluna "pdf" ->
# agora tem essas informacoes, por isso dropar "href" e "href formula".
datasetMenor.drop(["href"], axis=1, inplace=True)
datasetMenor.drop(["href formula"], axis=1, inplace=True)
datasetMenor

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,,
2.0,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001
3.0,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002
4.0,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003
5.0,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004
6.0,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299.0,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04
6300.0,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08
6301.0,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09


In [16]:
#os mesmos valores dessa coluna estao nas colunas case_number.1 e case_number.2
datasetMenor.drop(["Case Number"], axis=1, inplace=True)

In [17]:
datasetMenor

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,
2.0,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001
3.0,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002
4.0,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003
5.0,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004
6.0,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6299.0,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04
6300.0,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08
6301.0,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09


In [18]:
datasetMenor["Year"].unique()

array([   0.,    5.,   77.,  500., 1543., 1554., 1555., 1580., 1595.,
       1642., 1617., 1637., 1638., 1700., 1703., 1721., 1723., 1733.,
       1738., 1742., 1748., 1755., 1749., 1751., 1753., 1758., 1764.,
       1767., 1771., 1776., 1779., 1780., 1783., 1784., 1785., 1786.,
       1787., 1788., 1791., 1792., 1797., 1800., 1801., 1802., 1803.,
       1804., 1805., 1896., 1807., 1808., 1810., 1811., 1812., 1815.,
       1816., 1817., 1818., 1819., 1822., 1823., 1825., 1826., 1827.,
       1828., 1829., 1830., 1831., 1832., 1834., 1835., 1836.,   nan,
       1837., 1839., 1840., 1841., 1842., 1844., 1845., 1846., 1847.,
       1848., 1849., 1850., 1851., 1852., 1853., 1855., 1856., 1857.,
       1858., 1859., 1860., 1861., 1862., 1863., 1864., 1865., 1866.,
       1867., 1868., 1869., 1870., 1871., 1872., 1873., 1874., 1875.,
       1876., 1877., 1878., 1879., 1880., 1881., 1882., 1883., 1884.,
       1885., 1886., 1887., 1888., 1889., 1890., 1891., 1892., 1893.,
       1894., 1895.,

In [19]:
#Limpar valores incorretos da coluna "Year"
datasetMenor.loc[(datasetMenor["Year"] == 5)] 
datasetMenor.loc[(datasetMenor["Year"] == 77)] 


datasetMenor.loc[datasetMenor["Year"].isnull()]
datasetMenor.loc[(datasetMenor["Year"].isnull()) & (datasetMenor["Country"] == "ENGLAND"), "Year"] = 1836
datasetMenor.loc[(datasetMenor["Year"].isnull())& (datasetMenor["Country"] == "AUSTRALIA"), "Year"] = 2017

In [20]:
datasetMenor["Year"].unique()

array([   0.,    5.,   77.,  500., 1543., 1554., 1555., 1580., 1595.,
       1642., 1617., 1637., 1638., 1700., 1703., 1721., 1723., 1733.,
       1738., 1742., 1748., 1755., 1749., 1751., 1753., 1758., 1764.,
       1767., 1771., 1776., 1779., 1780., 1783., 1784., 1785., 1786.,
       1787., 1788., 1791., 1792., 1797., 1800., 1801., 1802., 1803.,
       1804., 1805., 1896., 1807., 1808., 1810., 1811., 1812., 1815.,
       1816., 1817., 1818., 1819., 1822., 1823., 1825., 1826., 1827.,
       1828., 1829., 1830., 1831., 1832., 1834., 1835., 1836., 1837.,
       1839., 1840., 1841., 1842., 1844., 1845., 1846., 1847., 1848.,
       1849., 1850., 1851., 1852., 1853., 1855., 1856., 1857., 1858.,
       1859., 1860., 1861., 1862., 1863., 1864., 1865., 1866., 1867.,
       1868., 1869., 1870., 1871., 1872., 1873., 1874., 1875., 1876.,
       1877., 1878., 1879., 1880., 1881., 1882., 1883., 1884., 1885.,
       1886., 1887., 1888., 1889., 1890., 1891., 1892., 1893., 1894.,
       1895., 1897.,

In [21]:
datasetMenor.loc[(datasetMenor["Year"] == 0) & (datasetMenor["case_number.2"])]
#Tentei achar uma associacao entre "Year" e "case_number.1 \ou/ .2", sem sucesso

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,
2.0,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001
3.0,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002
4.0,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003
5.0,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004
6.0,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122.0,Before 1939,0.0,Unprovoked,CANADA,NaN,Grand Banks,Fishing,Joe Folsom,M,NaN,Arm bitten,N,NaN,NaN,"C.E.Russell, pp. 310-311",http://sharkattackfile.net/spreadsheets/pdf_di...,ND-0154,ND-0154
123.0,Ca. 725 B.C.,0.0,Sea Disaster,ITALY,Tyrrhenian Sea,Krater found during excavations at Lacco Ameno...,Shipwreck,males,M,NaN,Depicts shipwrecked sailors attacked by a sha...,Y,NaN,NaN,"V.M. Coppleson (1958), p.262, et al",http://sharkattackfile.net/spreadsheets/pdf_di...,0000.0725,0000.0725
124.0,493 B.C.,0.0,Sea Disaster,GREECE,Off Thessaly,NaN,Shipwrecked Persian Fleet,males,M,NaN,Herodotus tells of sharks attacking men in the...,Y,NaN,NaN,Herodotus (485 - 425 B.C.),http://sharkattackfile.net/spreadsheets/pdf_di...,0000.0493,0000.0493


In [22]:
datasetMenor["Sex"].unique()

array(['M', 'F', nan, 'N', '.', 'lli', 'M '], dtype=object)

In [23]:
#Usar coluna "Name" para descobrir o "Sex"
datasetMenor.loc[(datasetMenor["Name"] == "male") & (datasetMenor["Sex"].isnull()), "Sex"] = "M"
datasetMenor.loc[(datasetMenor["Name"] == "female") & (datasetMenor["Sex"].isnull()), "Sex"] = "F"
# Arrumar valores de "Sex" usando unique()
datasetMenor.loc[(datasetMenor["Sex"] == "N"), "Sex"] = "M"
datasetMenor.loc[(datasetMenor["Sex"] == "lli"), "Sex"] = "M"
datasetMenor.loc[(datasetMenor["Sex"] == "M "), "Sex"] = "M"
datasetMenor.loc[(datasetMenor["Sex"] == "."), "Sex"] = np.NaN
datasetMenor.loc[(datasetMenor["Sex"].isnull())]
# Quando o "sex" é nulo parece que a maioria dos ataques foram contra barcos.

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,
16.0,Before 1917,0.0,Unprovoked,FIJI,Moala Island,NaN,Wreck of large double sailing canoe,20 Fijians,NaN,NaN,"FATAL, 18 people were killed by sharks, 2 sur...",Y,NaN,NaN,"Fijian Society papers presented April 17, 1918...",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0015,ND.0015
25.0,Between 1918 & 1939,0.0,Unprovoked,REUNION,Saint-Denis,Barachois,Swimming,NaN,NaN,NaN,FATAL,Y,NaN,NaN,G. Van Grevelynghe,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0024,ND.0024
44.0,1941-1945,0.0,Sea Disaster,NaN,NaN,NaN,A group of survivors on a raft for 17-days,C.,NaN,NaN,"FATAL, shark leapt into raft and bit the man w...",Y,Late afternoon,1.2 m [4'] shark,"G.A. Llano in Airmen Against the Sea, p.69",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0044,ND.0044
66.0,"No date, Before 1963",0.0,Unprovoked,SINGAPORE,NaN,"Keppel Harbor, 2 miles from Singapore city ce...",Swimming,NaN,NaN,NaN,Recovered,N,NaN,NaN,"V.M. Coppleson (1958), p.266",http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0073,ND.0073
91.0,Before 2004,0.0,Boat,MOZAMBIQUE,Inhambane Province,Off Inhambane,Fishing,"4.8-metre skiboat, Occupants: Rod Salm & 4 fri...",NaN,NaN,"No injury to occupants, shark bumped boat",N,NaN,Whale shark,South African Shark Attack File,http://sharkattackfile.net/spreadsheets/pdf_di...,ND-0107,ND-0107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149.0,17-Apr-2017,2017.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",NaN,NaN,NaN,NaN,Minor bite to the foot,N,Afternoon,NaN,"Daytona Beach News-Journal, 4/17/2017",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.17.b,2017.04.17.b
6179.0,02-Jul-2017,2017.0,Invalid,COMOROS,Anjouan,Moya,Fishing,NaN,NaN,NaN,"Skull found in shark, a probable drowning & sc...",NaN,NaN,Shark involvement prior to death not confirmed,"Linfo, 7/3/2017",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.07.02,2017.07.02
6217.0,Sep-2017,2017.0,Boating,AUSTRALIA,Westerm Australia,Esperance,Fishing,NaN,NaN,NaN,"sharks rammed boats, no injury to occupants",N,NaN,"White shark, 3.5m","B. Myatt, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.14,2017.09.14


In [24]:
datasetMenor["Sex"].unique()

array(['M', 'F', nan], dtype=object)

In [25]:
datasetMenor["Age"].unique()

array(['15', nan, '16', '13 or 14', '50', '6', '25', '36', '27', '19',
       '21', '20', '14', 'young', '22', '17', '52', '12', '29', '7 or 8',
       '1', '2½', '18', '28', '26', '13', '35', '40', '39', '30', '3',
       '11', '23', '60', '37', '33', '24', '9', 'Teen', '10', '8',
       '31 or 33', '55', '64', '7', '43', '10 or 12', '41', '38', '47',
       '54', '?    &   14', 'A.M.', '70', '78', '  ', '42', '50s', '49',
       '45', '36 & 23', '32', '34', '5', '31', '53', '9 or 10', 'Both 11',
       '48', 'F', '68', '8 or 10', '17 & 16', '46', '"young"', '56', '81',
       ' 43', 'MAKE LINE GREEN', '61', '62', '2 to 3 months', '57', '59',
       '33 & 26', '34 & 19', '13 or 18', '"middle-age"', 'X', '17 & 35',
       '50 & 30', '30 & 32', '44', '21, 34,24 & 35', '58',
       '37, 67, 35, 27,  ? & 27', '51', '25 or 28', '33 & 37', '(adult)',
       '23 & 26', '25 to 35', '9 months', '? & 19', '63', '9 & 12', '65',
       'adult', '18 to 22', '66', '>50', '20s', '20 ', '21 or 26', '

In [26]:
datasetMenor.loc[(datasetMenor["Age"] == "Y")] # Mesmo valor todas colunas
datasetMenor.drop(index=186, inplace=True)
# categorizar "Age" dos dados mais sujo em ranges. Depois fazer com o resto dos "Age"s.
datasetMenor.loc[datasetMenor["Age"] == 'F', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == ' 43', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == 'MAKE LINE GREEN', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == "A.M.", "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == "  ", "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == '(adult)', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == 'adult', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == ' ', "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == "\xa0 ", "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == "X", "Age"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Age"] == '2 to 3 months', "Age"] = " < 1"
datasetMenor.loc[datasetMenor["Age"] == '9 months', "Age"] = " < 1"
datasetMenor.loc[datasetMenor["Age"] == '18 months', "Age"] = "1"
datasetMenor.loc[datasetMenor["Age"] == "6½", "Age"] = "2 to 6"
datasetMenor.loc[datasetMenor["Age"] == "2½", "Age"] = "2 to 6"
datasetMenor.loc[datasetMenor["Age"] == '"young"', "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == "young", "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == "9 or 10", "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == "8 or 10", "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == "10 or 12", "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == 'Both 11', "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == 'Both 11', "Age"] = "7 to 12"
datasetMenor.loc[datasetMenor["Age"] == "Teens", "Age"] = "13 to 17"
datasetMenor.loc[datasetMenor["Age"] == "teen", "Age"] = "13 to 17"
datasetMenor.loc[datasetMenor["Age"] == "Teen", "Age"] = "13 to 17"
datasetMenor.loc[datasetMenor["Age"] == "?    &   14", "Age"] = "13 to 17"
datasetMenor.loc[datasetMenor["Age"] == '13 or 18', "Age"] = "13 to 17"
datasetMenor.loc[datasetMenor["Age"] == 'mid-20s', "Age"] = "23 to 29"
datasetMenor.loc[datasetMenor["Age"] == "20s", "Age"] = "23 to 29"
datasetMenor.loc[datasetMenor["Age"] == '"middle-age"', "Age"] = "30 to 39"
datasetMenor.loc[datasetMenor["Age"] == "31 or 33", "Age"] = "30 to 39"
datasetMenor.loc[datasetMenor["Age"] == "30s", "Age"] = "30 to 39"
datasetMenor.loc[datasetMenor["Age"] == "mid-30s", "Age"] = "30 to 39"
datasetMenor.loc[datasetMenor["Age"] == 'Ca. 33', "Age"] = "30 to 39"
datasetMenor.loc[datasetMenor["Age"] == "40s", "Age"] = "40 to 49"
datasetMenor.loc[datasetMenor["Age"] == '50s', "Age"] = "50 to 59"
datasetMenor.loc[datasetMenor["Age"] == '>50', "Age"] = "50 to 59"
datasetMenor.loc[datasetMenor["Age"] == 'Elderly'] = "60+"
datasetMenor.loc[datasetMenor["Age"] == "60's", "Age"] = "60+"
datasetMenor.loc[datasetMenor["Age"] == "60s", "Age"] = "60+"

In [27]:
datasetMenor["Age"].unique()

array(['15', nan, '16', '13 or 14', '50', '6', '25', '36', '27', '19',
       '21', '20', '14', '7 to 12', '22', '17', '52', '12', '29',
       '7 or 8', '1', '2 to 6', '18', '28', '26', '13', '35', '40', '39',
       '30', '3', '11', '23', '60', '37', '33', '24', '9', '13 to 17',
       '10', '8', '30 to 39', '55', '64', '7', '43', '41', '38', '47',
       '54', 'UNKNOWN', '70', '78', '42', '50 to 59', '49', '45',
       '36 & 23', '32', '34', '5', '31', '53', '48', '68', '17 & 16',
       '46', '56', '81', '61', '62', ' < 1', '57', '59', '33 & 26',
       '34 & 19', '17 & 35', '50 & 30', '30 & 32', '44', '21, 34,24 & 35',
       '58', '37, 67, 35, 27,  ? & 27', '51', '25 or 28', '33 & 37',
       '23 & 26', '25 to 35', '? & 19', '63', '9 & 12', '65', '18 to 22',
       '66', '23 to 29', '20 ', '21 or 26', '45 ', '74 ', '75', '60+',
       '77', '67', '87', '16 to 18', '32 & 30', '69', '71', '20?', ' 28',
       '7      &    31', ' 30', '23 & 20', '73', '33 or 37', '21 & ?',
       '3

In [28]:
datasetMenor["Fatal"].unique()

array(['Y', 'N', 'UNKNOWN', nan, 'N ', ' N', '60+', '2017', 'M'],
      dtype=object)

In [29]:
datasetMenor.loc[(datasetMenor["Fatal"] == "M"), "Fatal"] = "UNKNOWN"
datasetMenor.loc[datasetMenor["Fatal"] == "N ", "Fatal"] = "N"
datasetMenor.loc[datasetMenor["Fatal"] == " N", "Fatal"] = "N"
datasetMenor.loc[datasetMenor["Fatal"] == "y", "Fatal"] = "Y"
datasetMenor.loc[datasetMenor["Fatal"] == "2017", "Fatal"] = "N"


In [30]:
datasetMenor["Fatal"].unique()

array(['Y', 'N', 'UNKNOWN', nan, '60+'], dtype=object)

In [31]:
datasetMenor.loc[(datasetMenor["Fatal"].isnull()) & (datasetMenor["Injury"])]

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,
71.0,"No date, Before 1963",0,Invalid,USA,Hawaii,"Portlock, Oahu",Diving,Val Valentine,M,NaN,A 4.3 m [14'] shark made threat display. No in...,NaN,NaN,Invalid,B. Sojka & D. Lloyd,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0081,ND.0081
139.0,Late 1600s Reported 1728,1642,Invalid,GUINEA,NaN,NaN,Went overboard,crew member of the Nieuwstadt,M,NaN,FATAL,NaN,NaN,Questionable,"History of the Pyrates, by D. Defoe, Vol. 2, p.28",http://sharkattackfile.net/spreadsheets/pdf_di...,1642.00.00.b,1642.00.00.b
145.0,1723,1723,Unprovoked,ROATAN,NaN,NaN,NaN,Philip Ashton,M,NaN,Struck on thigh,NaN,NaN,NaN,"C.Moore, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,1723.00.00,1723.00.00
146.0,1733,1733,Invalid,ICELAND,Bardestrand,Talkknefiord,NaN,NaN,NaN,NaN,"Partial hominid remains recovered from shark, ...",NaN,NaN,Shark involvement prior to death unconfirmed,E. Olafsen,http://sharkattackfile.net/spreadsheets/pdf_di...,1733.00.00,1733.00.00
156.0,1767,1767,Invalid,FRANCE,Côte d'Azur,St. Tropez,Bathing,Samuel Matthews,M,NaN,Lacerations to arm & leg,NaN,NaN,Description of shark does not ring true,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,1767.00.00,1767.00.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6205.0,26-Aug-2017,2017,Invalid,SPAIN,Castellón,Grao de Moncofa,Swimming,female,F,11,Lacerations to left foot,NaN,Midday,Shark involvement questionable,"El Periodico Mediterraneo, 8/27/2017",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.08.26.b,2017.08.26.b
6219.0,15-Sep-2017,2017,Invalid,SOUTH AFRICA,Western Cape Province,Hawston,Scuba Diving,Wayon Love,M,25,"FATAL, but death was probably due to drowning",NaN,Afternoon,NaN,"Ground Up, 9/20/2017",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.15.b,2017.09.15.b
6224.0,24-Sep-2017,2017,Invalid,USA,New York,Rockaway,Surfing,Michah Behrend,M,33,"Lacerations to right ankle, foot & toe",NaN,15h30,Questionable,"S. Curatolo-Wageman, GSAF",http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.24.b,2017.09.24.b


In [32]:
datasetMenor.loc[(datasetMenor["Fatal"].isnull()) & ("FATAL" in datasetMenor["Injury"])]

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species,Investigator or Source,pdf,case_number.1,case_number.2
original_order,,,,,,,,,,,,,,,,,,


In [33]:
datasetMenor.to_csv(r"SharkDataCleaning.csv")